# Future Email Connections

**Note**: This problem is one of the assignments in [`Applied Social Network Analysis in Python`](https://www.coursera.org/learn/python-social-network-analysis).

**Goal**: Predict the employees who will more likely have an email exchange in the future. 

**Problem statement**: It's not indicated in the problem. This specific use case can be used for organizational analysis. 

**What we have**: 

- company's email network
    - nodes: employees
    - edges: having at least one email exchange between employees
- "future" connections data:
    - connections that were established after the creation of the email network
    - values: 1-connected, 0-not connected
    
Outline:
1. Importing packages and initial preprocessing
2. [Feature engineering](#sec_fe)
3. [Modeling](#sec_mod)
4. [Assessing models](#sec_am)
5. [Feature analysis](#sec_fa)

Note: Pipeline components and different parameters were not exhausted. Again, the purpose of this is to establish a structure where other collaborators can work on top. 

## Import packages

In [1]:
import os

import pandas as pd
import networkx as nx

import shap

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer
from sklearn.pipeline import FeatureUnion, Pipeline

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier

## User-defined parameters

In [2]:
# data path params
DATA_DIR = os.path.join(os.pardir, 'data')

GRAPH_FILE_NAME = 'email_prediction.txt' # company email network
CON_FILE_NAME = 'Future_Connections.csv' # future connections data

graph_file_path = os.path.join(DATA_DIR, GRAPH_FILE_NAME)
con_file_path = os.path.join(DATA_DIR, CON_FILE_NAME)


# graph params
community_name = 'Department'
response = 'connected'


# model params
random_state = 3
n_jobs = -1 # used generally throughout the code
cv = 5
scoring='roc_auc' # grid search evaluation metric

## Loading the Data

In [3]:
# company email network
graph = nx.read_gpickle(graph_file_path)

# future connections data
con_df = pd.read_csv(con_file_path, index_col=0, converters={0: eval})

#### Let's check the connections dataframe

In [4]:
con_df.head(3)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0


Fixing the indices and column names

In [5]:
con_df.reset_index(inplace=True)
con_df.rename(columns={'index': 'edge', 'Future Connection': response}, inplace=True)

In [6]:
con_df.head(3)

,edge,connected
0,"(6, 840)",0.0
1,"(4, 197)",0.0
2,"(620, 979)",0.0


Sample intances

In [7]:
pd.concat([
    con_df[con_df[response] == 1].head(2),
    con_df[con_df[response] == 0].head(2)
])

,edge,connected
5,"(97, 226)",1.0
16,"(342, 473)",1.0
0,"(6, 840)",0.0
1,"(4, 197)",0.0


Distribution of connections that were established

In [8]:
con_df[response].value_counts()

0.0    337002
1.0     29332
Name: connected, dtype: int64

Dropping instances with null `connected` values

In [9]:
con_df.isnull().sum()

edge              0
connected    122112
dtype: int64

In [10]:
con_df.dropna(subset=[response], inplace=True)

In [11]:
con_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366334 entries, 0 to 366333
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   edge       366334 non-null  object 
 1   connected  366334 non-null  float64
dtypes: float64(1), object(1)
memory usage: 8.4+ MB


#### Let's check some graph details

Graph type

In [12]:
type(graph)

networkx.classes.graph.Graph

In [13]:
graph.is_directed()

False

Basic information

In [14]:
print(nx.info(graph))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


Edges and nodes

In [15]:
graph.edges()[0:3]

[(0, 1), (0, 17), (0, 316)]

In [16]:
graph.nodes()[0:3]

[0, 1, 2]

In [17]:
graph.nodes(data=True)[0:3]

[(0, {'Department': 1, 'ManagementSalary': 0.0}),
 (1, {'Department': 1, 'ManagementSalary': nan}),
 (2, {'Department': 21, 'ManagementSalary': nan})]

The attributes are 'Department' and 'ManagementSalary'. We're only concerned with the 'Deparment' attribute since this will be used as community infomration. 

## Splitting the data

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    con_df.drop(labels=[response], axis=1),
    con_df[response],
    test_size=0.2,
    random_state=random_state,
    stratify=con_df[[response]]
)

<a id='sec_fe'></a>
## Feature Engineering

This section focuses on building custom transformers for extracting:
* purely edge features (`edge features`)
* community-based edge features (`com features`)

In this initial run, we'll explore the performance of the models trained on: only `edge features`, only `com features`, and both. 

If there are additional feature extractions discovered, just append a key-value pair of the function name and the function itself to `valid_extractors`. Then, add the function name when using the custom transformer. Note: this assumes that the new function follows the conventional syntax of the currently implemented extractors.

#### Custom Transformer: purely edge features

In [19]:
class EdgeFeaturesTransformer(BaseEstimator, TransformerMixin):
    
    valid_extractors = {
        'jaccard': nx.jaccard_coefficient, 
        'r_alloc': nx.resource_allocation_index,
        'p_attach': nx.preferential_attachment,
        # add feature extractors here
    }


    def __init__(self, graph, func_names):
        super().__init__()
        
        assert sum([(func_name not in self.valid_extractors.keys()) for func_name in func_names]) == 0,\
            f'invalid graph function set, valid options are {valid_extractors.keys()}'
        
        self.func_names = func_names
        self.graph = graph
    

    def fit(self, X, y=None):
        return self

        
    def transform(self, X, y=None):
        
        result_df = pd.DataFrame()

        for func_name in self.func_names:
            func = self.valid_extractors[func_name]
            computed_df = pd.DataFrame(
                    [i[2] for i in func(graph, X['edge'])], 
                    columns=[func_name]
                )
            result_df = pd.concat([result_df, computed_df], axis=1)
        
            
        return result_df
    
    def get_feature_names(self):
        return self.func_names

#### Custom Transformer: community-based edge features 

In [20]:
class ComFeaturesTransformer(BaseEstimator, TransformerMixin):
    
    valid_extractors = {
        'cn_sh': nx.cn_soundarajan_hopcroft,
        'ra_sh': nx.ra_index_soundarajan_hopcroft,
    }


    def __init__(self, graph, func_names, community_name):
        super().__init__()
        
        assert sum([(func_name not in self.valid_extractors.keys()) for func_name in func_names]) == 0,\
            f'invalid graph function set, valid options are {valid_extractors.keys()}'
        
        self.func_names = func_names
        self.community_name = community_name
        self.graph = graph
    

    def fit(self, X, y=None):
        return self

        
    def transform(self, X, y=None):
        
        result_df = pd.DataFrame()

        for func_name in self.func_names:
            func = self.valid_extractors[func_name]
            
            values = list(func(self.graph, community=self.community_name))
            
            node_value_dict = {(node_a, node_b): value for node_a, node_b, value in values}
            node_value_dict_r = {(node_b, node_a): value for node_a, node_b, value in values}
            node_value_dict.update(node_value_dict_r)
            
            computed_df = X['edge'].map(lambda x:node_value_dict[x])
            
            result_df = pd.concat([result_df, computed_df], axis=1)
        
            
        return result_df
    

    def get_feature_names(self):
        return self.func_names

<a id='sec_mod'></a>
## Modeling

### Building pipelines

#### Transformers

In [21]:
edge_transformers = make_column_transformer(
    (EdgeFeaturesTransformer(graph, ['jaccard', 'r_alloc', 'p_attach']), ['edge']),
    # add other transformers here
    n_jobs=n_jobs
)

com_transformers = make_column_transformer(
    (ComFeaturesTransformer(graph, ['cn_sh', 'ra_sh'], community_name), ['edge']),
    n_jobs=n_jobs
)

#### Model

Settling with random forest classifier only for now. Again, the focus is more on the feature engineering part. If there are other models to be considered, just add them in when building the [pipelines](#subsec_pipe). When adding new models, also add the hyper-parameter grid values for that specific model under this [subsection](#subsec_grid_param). 

In [22]:
rf_clf = RandomForestClassifier(
    random_state=random_state,
    warm_start=True
)

<a id='subsec_pipe'></a>
#### Pipelines

In [23]:
# pipeline utilizing only the edge features
pipeline_e = Pipeline([
    ('preprocessor', edge_transformers),
    ('rf_clf', rf_clf) # add other models here
]) 

# pipeline utilizing only the community-based edge features
pipeline_c = Pipeline([
    ('preprocessor', com_transformers),
    ('rf_clf', rf_clf)
]) 

# pipeline utilizing both.
union = FeatureUnion([
    ('edge', edge_transformers),
    ('com', com_transformers)
])

pipeline_ec = Pipeline([
    ('preprocessor', union),
    ('rf_clf', rf_clf)
]) 

#### Visualize the pipelines

In [24]:
from sklearn import set_config

set_config(display='diagram')

In [25]:
pipeline_e

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('edgefeaturestransformer',
                                                  EdgeFeaturesTransformer(func_names=['jaccard',
                                                                                      'r_alloc',
                                                                                      'p_attach'],
                                                                          graph=<networkx.classes.graph.Graph object at 0x7fcb78cc6128>),
                                                  ['edge'])])),
                ('rf_clf',
                 RandomForestClassifier(random_state=3, warm_start=True))])

In [26]:
pipeline_c

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('comfeaturestransformer',
                                                  ComFeaturesTransformer(community_name='Department',
                                                                         func_names=['cn_sh',
                                                                                     'ra_sh'],
                                                                         graph=<networkx.classes.graph.Graph object at 0x7fcb78cc6128>),
                                                  ['edge'])])),
                ('rf_clf',
                 RandomForestClassifier(random_state=3, warm_start=True))])

In [27]:
pipeline_ec

Pipeline(steps=[('preprocessor',
                 FeatureUnion(transformer_list=[('edge',
                                                 ColumnTransformer(n_jobs=-1,
                                                                   transformers=[('edgefeaturestransformer',
                                                                                  EdgeFeaturesTransformer(func_names=['jaccard',
                                                                                                                      'r_alloc',
                                                                                                                      'p_attach'],
                                                                                                          graph=<networkx.classes.graph.Graph object at 0x7fcb78cc6128>),
                                                                                  ['edge'])])),
                                                ('com',
                                                 ColumnTransformer(n_jobs=-1,
                                                                   transformers=[('comfeaturestransformer',
                                                                                  ComFeaturesTransformer(community_name='Department',
                                                                                                         func_names=['cn_sh',
                                                                                                                     'ra_sh'],
                                                                                                         graph=<networkx.classes.graph.Graph object at 0x7fcb78cc6128>),
                                                                                  ['edge'])]))])),
                ('rf_clf',
                 RandomForestClassifier(random_state=3, warm_start=True))])

### Selecting model and model hyper-parameters

<a id='subsec_grid_param'></a>
#### Defining hyper-parameter candidates

In [28]:
# define the hyper-parameters
param_grid = dict(
    rf_clf__n_estimators=[10, 100],
    rf_clf__max_depth=[2, 10]
    # rf_clf__max_features = [] - this should be explored when we have high-dimensional data
)

#### Modeling and selecting model based on defined score

In [29]:
grid_e = GridSearchCV(
    estimator=pipeline_e,
    param_grid=param_grid,
    scoring=scoring,
    n_jobs=n_jobs,
    verbose=8,
    cv=cv
)

grid_c = GridSearchCV(
    estimator=pipeline_c,
    param_grid=param_grid,
    scoring=scoring,
    n_jobs=n_jobs,
    verbose=8,
    cv=cv
)

grid_ec = GridSearchCV(
    estimator=pipeline_ec,
    param_grid=param_grid,
    scoring=scoring,
    n_jobs=n_jobs,
    verbose=8,
    cv=cv
)

Defining `grids` dictionary to be more organized

In [30]:
grids = {
    'edge':  grid_e,
    'com':   grid_c,
    'union': grid_ec
}

Sanity check of the whole pipeline (union pipeline)

In [31]:
grids['union']

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        FeatureUnion(transformer_list=[('edge',
                                                                        ColumnTransformer(n_jobs=-1,
                                                                                          transformers=[('edgefeaturestransformer',
                                                                                                         EdgeFeaturesTransformer(func_names=['jaccard',
                                                                                                                                             'r_alloc',
                                                                                                                                             'p_attach'],
                                                                                                                                 graph=<networkx.classes.graph.Graph object at 0x7fcb78cc6128>),
                                                                                                         ['edge'])])),
                                                                       ('com',
                                                                        ColumnTransformer(n_jobs=-1,
                                                                                          transf...restransformer',
                                                                                                         ComFeaturesTransformer(community_name='Department',
                                                                                                                                func_names=['cn_sh',
                                                                                                                                            'ra_sh'],
                                                                                                                                graph=<networkx.classes.graph.Graph object at 0x7fcb78cc6128>),
                                                                                                         ['edge'])]))])),
                                       ('rf_clf',
                                        RandomForestClassifier(random_state=3,
                                                               warm_start=True))]),
             n_jobs=-1,
             param_grid={'rf_clf__max_depth': [2, 10],
                         'rf_clf__n_estimators': [10, 100]},
             scoring='roc_auc', verbose=8)

Fitting the data

In [32]:
for grid_name, grid in grids.items():
    print(f'Performing gridsearchCV: {grid_name}')
    grid.fit(X_train, y_train)

Performing gridsearchCV: edge
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Performing gridsearchCV: com
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Performing gridsearchCV: union
Fitting 5 folds for each of 4 candidates, totalling 20 fits


<a id='sec_am'></a>
# Assessing the models 


#### model parameters and scores

In [33]:
for grid_name, grid in grids.items():
    # TODO: add storage of results for further hyper-parameter tuning
    
    grid.predict(X_test)
    print(f"""
    pipeline:          {grid_name}
    best params:       {grid.best_params_}
    mean test score:   {grid.cv_results_['mean_test_score']}
    std test score:    {grid.cv_results_['std_test_score']}
    """)


    pipeline:          edge
    best params:       {'rf_clf__max_depth': 10, 'rf_clf__n_estimators': 100}
    mean test score:   [0.89599048 0.90057569 0.90898618 0.91013192]
    std test score:    [0.00382839 0.00392864 0.0038263  0.00382244]
    

    pipeline:          com
    best params:       {'rf_clf__max_depth': 10, 'rf_clf__n_estimators': 100}
    mean test score:   [0.88760489 0.89057226 0.91321218 0.91335714]
    std test score:    [0.00235675 0.00252909 0.00344154 0.0033958 ]
    

    pipeline:          union
    best params:       {'rf_clf__max_depth': 10, 'rf_clf__n_estimators': 100}
    mean test score:   [0.89576545 0.90206909 0.91328739 0.91399391]
    std test score:    [0.00324586 0.00341265 0.00353051 0.00365843]
    


Analyze the results. Perform the necessary changes/additions to [feature engineering](#sec_fe) tasks or [hyper-parameters](#subsec_grid_pram) improve the model. Or expore other [models](#sec_mod). 

TODO:
- add test results
- store results data
- store models

<a id='sec_fa'></a>
# Feature importance and instance analysis

TODO: 
- Implement shap sampling
- Make this Google Colab compatible
- Instance analysis

In [ ]:
# assign the model to analyze 
model = grids['union']

In [ ]:
X = model.best_estimator_.named_steps['preprocessor'].fit_transform(X_train)
explainer = shap.KernelExplainer(model.best_estimator_.named_steps['rf_clf'].predict_proba, X)

In [ ]:
shap_values = explainer.shap_values(X)